In [1]:
import pandas as pd
import numpy as np
import re
import Levenshtein

In [2]:
#import mine data
#primary data
mine_full = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-April-2024 (1).xlsx", sheet_name="Global Coal Mine Tracker (Non-C")
#supplement data
mine_sup1 = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-September-2024-Supplement-v2 (2).xlsx", sheet_name="Historical Production (non-Chin")
mine_sup2 =pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-December-2024-Supplement-Historical-Production-from-2018-to-2023.xlsx", sheet_name="Historical Production(non-China")

In [3]:
#import plant data
plant_full = pd.read_excel("./Data/new coal/Global-Coal-Plant-Tracker-January-2025.xlsx", sheet_name="Units")

#correct the estimated capacity with capacity factor
plant_full['Est_capacity (MW)']=plant_full['Capacity (MW)']*plant_full['Capacity factor']

#import terminal data
#note: the capacity of terminal dataset may be overestimated, because some terminals not only receive 'coal' but 'all cargo'
terminal_full = pd.read_excel("./Data/new coal/Global-Coal-Terminals-Tracker-December-2024.xlsx", sheet_name="Terminals")

In [4]:
mine = mine_full[mine_full['Status'] != 'Cancelled']
plant = plant_full[plant_full['Status'] != 'cancelled']
terminal = terminal_full[terminal_full['Status'] != 'Cancelled']

In [5]:
mine = mine[['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)', 'Owners']]
plant = plant[['GEM location ID', 'Plant name', 'Latitude', 'Longitude','Country/Area', 'Est_capacity (MW)', 'Owner']]
terminal = terminal[['GEM Terminal ID', 'Coal Terminal Name', 'Latitude', 'Longitude','Country/Area', 'Capacity (Mt)','Owner']]

In [6]:
mine.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production', 'firm']
plant.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production', 'firm']
terminal.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production', 'firm']

In [7]:
mine = mine[mine['country'] == 'India']
plant = plant[plant['country'] == 'India']
terminal = terminal[terminal['country'] == 'India']

In [8]:
total_data = pd.concat([mine, plant, terminal], ignore_index=True)

In [9]:
'''
#may further filter by 'status == 'operating'
mine_pri = mine_pri[mine_pri['Status'] == 'Operating']
plant = plant[plant['Status'] == 'operating']
terminal = terminal[terminal['Status'] == 'Operating']
'''

"\n#may further filter by 'status == 'operating'\nmine_pri = mine_pri[mine_pri['Status'] == 'Operating']\nplant = plant[plant['Status'] == 'operating']\nterminal = terminal[terminal['Status'] == 'Operating']\n"

## Sheet 1: Assets

In [10]:
mine_assets = mine[['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']]
plant_assets = plant[['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']]
terminal_assets = terminal[['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']]

In [11]:
mine_assets['product'] = 'coal'
plant_assets['product'] = 'power'
terminal_assets['product'] = 'coal'#may be incorrect because some are 'all cargo'

In [12]:
'''
mine_assets = mine_pri[['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)']]
plant_assets = plant[['GEM location ID', 'Plant name', 'Latitude', 'Longitude','Country/Area', 'Est_capacity (MW)' ]]
terminal_assets = terminal[['GEM Terminal ID', 'Coal Terminal Name', 'Latitude', 'Longitude','Country/Area', 'Capacity (Mt)']]

mine_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
plant_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
terminal_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
'''

def process_assets(df):
    # Split data: rows where production = '*' and other numeric rows that can be aggregated
    star_rows = df[df["production"] == '*']  # Rows where production value is '*'
    numeric_rows = df[df["production"] != '*'].copy()  # Other rows that can be aggregated

    # Ensure production is a numeric type
    numeric_rows["production"] = pd.to_numeric(numeric_rows["production"], errors="coerce").fillna(0)

    # Perform groupby aggregation only on numeric rows
    grouped_numeric = numeric_rows.groupby("asset_id", as_index=False).agg({
        "name": "first",
        "latitude": "first",
        "longitude": "first",
        "country": "first",
        "production": "sum",
        "product": "first"
    })

    # Merge the aggregated groupby results with the original rows where production = '*'
    return pd.concat([grouped_numeric, star_rows], ignore_index=True)

# Process each dataset
mine_assets = process_assets(mine_assets)
plant_assets = process_assets(plant_assets)
terminal_assets = process_assets(terminal_assets)

# Combine all asset data
total_assets = pd.concat([mine_assets, plant_assets, terminal_assets], ignore_index=True)

# Output final result
total_assets

,asset_id,name,latitude,longitude,country,production,product
0,M0464,Adasa Coal Mine,21.347003,78.953545,India,0.61,coal
1,M0465,Adriyala Coal Mine,18.6648,79.579,India,1.54,coal
2,M0466,AKK Coal Mine,23.7905,85.9198,India,4.25,coal
3,M0467,Amadand Coal Mine,23.157942,82.061533,India,0.87,coal
4,M0468,Amalgamated Keshalpur-West Mudidih (AKWMC) Coa...,23.8078,86.3221,India,4.1,coal
...,...,...,...,...,...,...,...
909,T1413,Port of Chennai Coal Terminal,13.085456,80.296283,India,8.0,coal
910,T1414,Haji Bunder Port Coal Terminal,19.59,73.41,India,1.8,coal
911,T1415,Visakhapatnam Port Adani Terminal,17.68333,83.2833,India,6.4,coal
912,T1432,Rewas Port Coal Terminal,18.79178,72.914995,India,66.0,coal


In [13]:
prod_index = total_assets.columns.get_loc('production')

total_assets.insert(prod_index, 'replacement_value', '')

In [14]:
total_assets

,asset_id,name,latitude,longitude,country,replacement_value,production,product
0,M0464,Adasa Coal Mine,21.347003,78.953545,India,,0.61,coal
1,M0465,Adriyala Coal Mine,18.6648,79.579,India,,1.54,coal
2,M0466,AKK Coal Mine,23.7905,85.9198,India,,4.25,coal
3,M0467,Amadand Coal Mine,23.157942,82.061533,India,,0.87,coal
4,M0468,Amalgamated Keshalpur-West Mudidih (AKWMC) Coa...,23.8078,86.3221,India,,4.1,coal
...,...,...,...,...,...,...,...,...
909,T1413,Port of Chennai Coal Terminal,13.085456,80.296283,India,,8.0,coal
910,T1414,Haji Bunder Port Coal Terminal,19.59,73.41,India,,1.8,coal
911,T1415,Visakhapatnam Port Adani Terminal,17.68333,83.2833,India,,6.4,coal
912,T1432,Rewas Port Coal Terminal,18.79178,72.914995,India,,66.0,coal


## Sheet 2: Firms

In [15]:
# Select necessary columns
mine_firms = mine[['asset_id', 'name', 'firm']]
plant_firms = plant[['asset_id', 'name', 'firm']]
terminal_firms = terminal[['asset_id', 'name', 'firm']]

In [16]:
# Concatenate all the dfs together
all_firms = pd.concat([mine_firms, plant_firms, terminal_firms], ignore_index=True)

In [17]:
all_firms # 1787 rows

,asset_id,name,firm
0,M0464,Adasa Coal Mine,Western Coalfields Ltd
1,M0465,Adriyala Coal Mine,The Singareni Collieries Co Ltd
2,M0466,AKK Coal Mine,Central Coalfields Ltd
3,M0467,Amadand Coal Mine,South Eastern Coalfields Ltd
4,M0467,Amadand Coal Mine,South Eastern Coalfields Ltd
...,...,...,...
1782,T1058,Tuticorin Port Coal Terminal,V.O. Chidambaranar Port Trust
1783,T1058,Tuticorin Port Coal Terminal,V.O. Chidambaranar Port Trust
1784,T1415,Visakhapatnam Port Adani Terminal,Adani Group
1785,T1177,Visakhapatnam Port Coal Terminal,Visakhapatnam Port Trust (VPT)


In [18]:
# Add more columns that will be needed
def add_firms_columns(df, owner_column, firm_type):

    firms = df[[owner_column]].rename(columns={owner_column: 'name'})
    
    # Adding columns
    firms['firm_id'] = np.nan
    firms['final_demand'] = np.nan
    firms['discount_rate'] = np.nan
    firms['unit_price'] = np.nan
    firms['margin'] = np.nan
    firms['leverage_ratio'] = np.nan
    firms['type'] = firm_type
    
    # Adjust order of columns
    firms = firms[['firm_id', 'name', 'final_demand', 'discount_rate', 
                   'unit_price', 'margin', 'leverage_ratio', 'type']]
    
    return firms

# Apply functions
total_firms = add_firms_columns(all_firms, 'firm', 'production')

# Remove duplicates
total_firms.drop_duplicates(subset=['name'], keep='first', inplace=True)

In [19]:
total_firms # 324 rows

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,NaN,Western Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
1,NaN,The Singareni Collieries Co Ltd,NaN,NaN,NaN,NaN,NaN,production
2,NaN,Central Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
3,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
5,NaN,Bharat Coking Coal Ltd,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
1778,NaN,Essar Group,NaN,NaN,NaN,NaN,NaN,production
1779,NaN,Shree Cement,NaN,NaN,NaN,NaN,NaN,production
1780,NaN,Tata Group,NaN,NaN,NaN,NaN,NaN,production
1782,NaN,V.O. Chidambaranar Port Trust,NaN,NaN,NaN,NaN,NaN,production


For some datasets, the shares of firms are indicated in the owner names, so we need to separate the firm names

In [20]:
# Filter rows where 'name' contains '['
filtered_df = total_firms[total_firms['name'].str.contains(r'\(', regex=True)]

# Display the filtered rows
filtered_df

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
544,NaN,Anuppur Thermal Energy (MP) Pvt Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
601,NaN,MB Power (Madhya Pradesh) Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
733,NaN,ACB (India) Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
957,NaN,Ind-Barath Energy (Utkal) Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
1194,NaN,Mirzapur Thermal Energy (UP) Pvt Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
1196,NaN,Pench Thermal Energy (MP) Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
1313,NaN,Aditya Birla Chemicals (India) Ltd [100%],NaN,NaN,NaN,NaN,NaN,production
1742,NaN,Mumbai Port Trust (MbPT),NaN,NaN,NaN,NaN,NaN,production
1745,NaN,Essar Group (74%); AM/NS (26%),NaN,NaN,NaN,NaN,NaN,production
1765,NaN,Mormugao Port Trust (MPT),NaN,NaN,NaN,NaN,NaN,production


## Format firm names

In [21]:
def process_firm_name(df):
    
    # Step 1: Split the 'name' column by ';' into two separate columns
    df[['name1', 'name2','name3']] = df['name'].str.split(r'[;,]', expand=True)

    # Step 2: Duplicate all other columns for each new entry
    df_expanded = df.drop(columns=['name'])  # Drop the original 'name' column

    # Step 3: Melt the 'name1' and 'name2' columns
    df_flat = pd.melt(df_expanded, id_vars=['firm_id', 'final_demand', 'discount_rate', 'unit_price', 'margin', 'leverage_ratio', 'type'], 
                      value_vars=['name1', 'name2','name3'], value_name='name')

    # Step 4: Remove rows with NaN values in 'name'
    df_flat = df_flat.dropna(subset=['name'])

    # Step 5: Move 'name' to the second column position
    df_flat.insert(1, 'name', df_flat.pop('name'))

    # Reset index for clean output
    df_flat.reset_index(drop=True, inplace=True)

    # Drop 'variable' column
    df_flat = df_flat.drop(columns=['variable'])

    return df_flat

# Apply the function
total_firms_flat = process_firm_name(total_firms)

In [22]:
# Display the result
total_firms_flat # 343 rows

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,NaN,Western Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
1,NaN,The Singareni Collieries Co Ltd,NaN,NaN,NaN,NaN,NaN,production
2,NaN,Central Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
3,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
4,NaN,Bharat Coking Coal Ltd,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
338,NaN,AM/NS (26%),NaN,NaN,NaN,NaN,NaN,production
339,NaN,Bothra Group,NaN,NaN,NaN,NaN,NaN,production
340,NaN,JSW Infrastructure Ltd.,NaN,NaN,NaN,NaN,NaN,production
341,NaN,Maharashtra Maritime Board,NaN,NaN,NaN,NaN,NaN,production


In [23]:
# In the 'firms' sheet, we don't need the share data, so remove them here
# Remove percentage patterns inside square brackets or parentheses
total_firms_flat['name'] = total_firms_flat['name'].str.replace(r'\[.*?\]', '', regex=True)  # Remove square brackets content
total_firms_flat['name'] = total_firms_flat['name'].str.replace(r'\(\d+(\.\d+)?%\)', '', regex=True)  # Remove percentage in parentheses
# Remove a dot at the end of the name (if it exists)
total_firms_flat['name'] = total_firms_flat['name'].str.replace(r'\.$', '', regex=True)
total_firms_flat['name'] = total_firms_flat['name'].str.strip()
total_firms_flat

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,NaN,Western Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
1,NaN,The Singareni Collieries Co Ltd,NaN,NaN,NaN,NaN,NaN,production
2,NaN,Central Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
3,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
4,NaN,Bharat Coking Coal Ltd,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
338,NaN,AM/NS,NaN,NaN,NaN,NaN,NaN,production
339,NaN,Bothra Group,NaN,NaN,NaN,NaN,NaN,production
340,NaN,JSW Infrastructure Ltd,NaN,NaN,NaN,NaN,NaN,production
341,NaN,Maharashtra Maritime Board,NaN,NaN,NaN,NaN,NaN,production


In [24]:
# Function to standardize company names
def standardize_company_name(name):
    # Remove "Ltd" and "Limited"
    name = re.sub(r'\bLtd\b', '', name)
    name = re.sub(r'\bLimited\b', '', name)
    
    # Replace "Co" with "Company"
    name = re.sub(r'\bCo\b', 'Company', name)
    
    # Replace "Corp" with "Corporation"
    name = re.sub(r'\bCorp\b', 'Corporation', name)
    
    # Remove duplicate "Company Company"
    name = re.sub(r'\bCompany\b\s*\bCompany\b', 'Company', name)
    
    # Remove any extra spaces
    name = name.strip()
    
    return name

# Apply the function to the 'Company Name' column
total_firms_flat['name'] = total_firms_flat['name'].apply(standardize_company_name)

In [25]:
total_firms_flat

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,NaN,Western Coalfields,NaN,NaN,NaN,NaN,NaN,production
1,NaN,The Singareni Collieries Company,NaN,NaN,NaN,NaN,NaN,production
2,NaN,Central Coalfields,NaN,NaN,NaN,NaN,NaN,production
3,NaN,South Eastern Coalfields,NaN,NaN,NaN,NaN,NaN,production
4,NaN,Bharat Coking Coal,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
338,NaN,AM/NS,NaN,NaN,NaN,NaN,NaN,production
339,NaN,Bothra Group,NaN,NaN,NaN,NaN,NaN,production
340,NaN,JSW Infrastructure,NaN,NaN,NaN,NaN,NaN,production
341,NaN,Maharashtra Maritime Board,NaN,NaN,NaN,NaN,NaN,production


In [26]:
total_firms_flat = total_firms_flat.drop_duplicates(subset=['name'])

In [27]:
# Function to find pairs of names with less than 3 letters difference
def find_similar_names(df):
    names = df['name'].tolist()  # Get list of names
    similar_names = []

    # Compare each pair of names
    for i in range(len(names)):
        for j in range(i + 1, len(names)):
            # Calculate the Levenshtein distance between the two names
            distance = Levenshtein.distance(names[i], names[j])
            # If the distance is less than 3, consider it similar
            if distance < 3 and distance > 0:
                similar_names.append((names[i], names[j], distance))

    return similar_names

# Find similar names with less than 3 letters difference
similar_names = find_similar_names(total_firms_flat)

# Print the results
for name1, name2, dist in similar_names:
    print(f"'{name1}' & '{name2}':  have a Levenshtein distance of {dist}")

'Western Coalfields' & 'Eastern Coalfields':  have a Levenshtein distance of 2
'South Eastern Coalfields' & 'North Eastern Coalfields':  have a Levenshtein distance of 2
'NTPC' & 'NMDC':  have a Levenshtein distance of 2
'Ambujia Cements' & 'Ambuja Cements':  have a Levenshtein distance of 1
'CG Natural Resources Pvt' & 'MH Natural Resources Pvt':  have a Levenshtein distance of 2
'GVK Energy' & 'GMR Energy':  have a Levenshtein distance of 2
'Kakinada Seaport' & 'Kakinada Seaports':  have a Levenshtein distance of 1


By manually inspecting the data, we also found there are `Gujarat Mineral Development Company` and `Gujarat Mineral Development Corporation`, however there is only info about `Gujarat Mineral Development Corporation` online, so we manually change it here. 

Also it's noticeable that `Ambujia Cements` and `Ambuja Cements` are indeed different firms.

In [28]:
# Manually replace company names
# comment out Gujarat Mineral Development because after filter out cancelled projects, it's not in dataset anymore
# total_firms_flat['name'] = total_firms_flat['name'].replace('Gujarat Mineral Development Company', 'Gujarat Mineral Development Corporation')
total_firms_flat['name'] = total_firms_flat['name'].replace('Kakinada Seaport', 'Kakinada Seaports')
total_firms_flat = total_firms_flat[total_firms_flat['name'] != '-']

/var/folders/3h/83jfd71114x78b1sj0_llf1m0000gp/T/ipykernel_4289/3391142284.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_firms_flat['name'] = total_firms_flat['name'].replace('Kakinada Seaport', 'Kakinada Seaports')


In [29]:
total_firms_flat = total_firms_flat.sort_values(by='name')
# Display the unique names
total_firms_flat # 280 rows

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
159,NaN,ACB (India),NaN,NaN,NaN,NaN,NaN,production
338,NaN,AM/NS,NaN,NaN,NaN,NaN,NaN,production
218,NaN,AM/NS Khopoli,NaN,NaN,NaN,NaN,NaN,production
96,NaN,AP Mineral Development Corporation,NaN,NaN,NaN,NaN,NaN,production
316,NaN,APM Terminals,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
221,NaN,Welspun Captive Power Generation,NaN,NaN,NaN,NaN,NaN,production
18,NaN,West Bengal Mineral Development and Trading Co...,NaN,NaN,NaN,NaN,NaN,production
24,NaN,West Bengal Power Development Corporation,NaN,NaN,NaN,NaN,NaN,production
0,NaN,Western Coalfields,NaN,NaN,NaN,NaN,NaN,production


In [30]:
unique_firms = total_firms_flat.reset_index(drop=True)
unique_firms = unique_firms.drop_duplicates(subset=['name'])
# Generate a list of firm IDs starting from F001 to Fxxx, based on the length of the DataFrame
unique_firms['firm_id'] = ['F' + str(i+1).zfill(3) for i in range(len(unique_firms))]
unique_firms

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,F001,ACB (India),NaN,NaN,NaN,NaN,NaN,production
1,F002,AM/NS,NaN,NaN,NaN,NaN,NaN,production
2,F003,AM/NS Khopoli,NaN,NaN,NaN,NaN,NaN,production
3,F004,AP Mineral Development Corporation,NaN,NaN,NaN,NaN,NaN,production
4,F005,APM Terminals,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
275,F275,Welspun Captive Power Generation,NaN,NaN,NaN,NaN,NaN,production
276,F276,West Bengal Mineral Development and Trading Co...,NaN,NaN,NaN,NaN,NaN,production
277,F277,West Bengal Power Development Corporation,NaN,NaN,NaN,NaN,NaN,production
278,F278,Western Coalfields,NaN,NaN,NaN,NaN,NaN,production


# Sheet 3: ownership

In [31]:
all_firms

,asset_id,name,firm
0,M0464,Adasa Coal Mine,Western Coalfields Ltd
1,M0465,Adriyala Coal Mine,The Singareni Collieries Co Ltd
2,M0466,AKK Coal Mine,Central Coalfields Ltd
3,M0467,Amadand Coal Mine,South Eastern Coalfields Ltd
4,M0467,Amadand Coal Mine,South Eastern Coalfields Ltd
...,...,...,...
1782,T1058,Tuticorin Port Coal Terminal,V.O. Chidambaranar Port Trust
1783,T1058,Tuticorin Port Coal Terminal,V.O. Chidambaranar Port Trust
1784,T1415,Visakhapatnam Port Adani Terminal,Adani Group
1785,T1177,Visakhapatnam Port Coal Terminal,Visakhapatnam Port Trust (VPT)


In [32]:
all_firms = all_firms.drop_duplicates(subset=['asset_id', 'firm'])
all_firms

,asset_id,name,firm
0,M0464,Adasa Coal Mine,Western Coalfields Ltd
1,M0465,Adriyala Coal Mine,The Singareni Collieries Co Ltd
2,M0466,AKK Coal Mine,Central Coalfields Ltd
3,M0467,Amadand Coal Mine,South Eastern Coalfields Ltd
5,M0468,Amalgamated Keshalpur-West Mudidih (AKWMC) Coa...,Bharat Coking Coal Ltd
...,...,...,...
1781,T1165,Tuna Tekra Port Coal Terminal,Adani Group
1782,T1058,Tuticorin Port Coal Terminal,V.O. Chidambaranar Port Trust
1784,T1415,Visakhapatnam Port Adani Terminal,Adani Group
1785,T1177,Visakhapatnam Port Coal Terminal,Visakhapatnam Port Trust (VPT)


In [33]:
ownership = all_firms[['asset_id', 'firm']]
ownership

,asset_id,firm
0,M0464,Western Coalfields Ltd
1,M0465,The Singareni Collieries Co Ltd
2,M0466,Central Coalfields Ltd
3,M0467,South Eastern Coalfields Ltd
5,M0468,Bharat Coking Coal Ltd
...,...,...
1781,T1165,Adani Group
1782,T1058,V.O. Chidambaranar Port Trust
1784,T1415,Adani Group
1785,T1177,Visakhapatnam Port Trust (VPT)


In [34]:
def assign_share(firm):
    if any(char.isdigit() for char in firm) or ';' in firm or ',' in firm:
        return ''  # Empty if condition is met
    else:
        return 1  # Share = 1 if no digits, ';' or ','

ownership['share1'] = ownership['firm'].apply(assign_share)

In [35]:
ownership

,asset_id,firm,share1
0,M0464,Western Coalfields Ltd,1
1,M0465,The Singareni Collieries Co Ltd,1
2,M0466,Central Coalfields Ltd,1
3,M0467,South Eastern Coalfields Ltd,1
5,M0468,Bharat Coking Coal Ltd,1
...,...,...,...
1781,T1165,Adani Group,1
1782,T1058,V.O. Chidambaranar Port Trust,1
1784,T1415,Adani Group,1
1785,T1177,Visakhapatnam Port Trust (VPT),1


In [36]:
def process_firm_name_ownership(df):
    
    # Step 1: Split the 'name' column by ';' into two separate columns
    df[['name1', 'name2','name3']] = df['firm'].str.split(r'[;,]', expand=True)

    # Step 2: Duplicate all other columns for each new entry
    df_expanded = df.drop(columns=['firm'])  # Drop the original 'name' column

    # Step 3: Melt the 'name1' and 'name2' columns
    df_flat = pd.melt(df_expanded, id_vars=['asset_id', 'share1'], 
                      value_vars=['name1', 'name2','name3'], value_name='firm')

    # Step 4: Remove rows with NaN values in 'name'
    df_flat = df_flat.dropna(subset=['firm'])

    # Step 5: Move 'name' to the second column position
    df_flat.insert(1, 'firm', df_flat.pop('firm'))

    # Reset index for clean output
    df_flat.reset_index(drop=True, inplace=True)

    # Drop 'variable' column
    df_flat = df_flat.drop(columns=['variable'])

    return df_flat

# Apply the function
ownership_flat = process_firm_name_ownership(ownership)

In [37]:
ownership_flat

,asset_id,firm,share1
0,M0464,Western Coalfields Ltd,1
1,M0465,The Singareni Collieries Co Ltd,1
2,M0466,Central Coalfields Ltd,1
3,M0467,South Eastern Coalfields Ltd,1
4,M0468,Bharat Coking Coal Ltd,1
...,...,...,...
905,T1160,Bothra Group,
906,T1167,JSW Infrastructure Ltd.,
907,T1432,Maharashtra Maritime Board,
908,T1382,Maharashtra Maritime Board,


In [38]:
def extract_percentage(firm):
    match = re.search(r'[\[\(](\d+(\.\d+)?%)[]\)]', firm)
    return match.group(1) if match else ''

# Apply the function to create the new column
ownership_flat['share2'] = ownership_flat['firm'].apply(extract_percentage)
ownership_flat

,asset_id,firm,share1,share2
0,M0464,Western Coalfields Ltd,1,
1,M0465,The Singareni Collieries Co Ltd,1,
2,M0466,Central Coalfields Ltd,1,
3,M0467,South Eastern Coalfields Ltd,1,
4,M0468,Bharat Coking Coal Ltd,1,
...,...,...,...,...
905,T1160,Bothra Group,,
906,T1167,JSW Infrastructure Ltd.,,
907,T1432,Maharashtra Maritime Board,,
908,T1382,Maharashtra Maritime Board,,


In [39]:
def convert_to_float(value):
    if isinstance(value, str):
        # Convert percentage to float if it exists
        if '%' in value:
            return float(value.replace('%', '')) / 100
        elif value == '':
            return 0.0  # Treat empty strings as 0
    return float(value) if value else 0.0

# Convert 'share1' and 'share2' columns to float
ownership_flat['share1'] = ownership_flat['share1'].apply(convert_to_float)
ownership_flat['share2'] = ownership_flat['share2'].apply(convert_to_float)

# Create a new column 'share' as the sum of 'share1' and 'share2'
ownership_flat['share'] = ownership_flat[['share1', 'share2']].sum(axis=1)
ownership_flat

,asset_id,firm,share1,share2,share
0,M0464,Western Coalfields Ltd,1.0,0.0,1.0
1,M0465,The Singareni Collieries Co Ltd,1.0,0.0,1.0
2,M0466,Central Coalfields Ltd,1.0,0.0,1.0
3,M0467,South Eastern Coalfields Ltd,1.0,0.0,1.0
4,M0468,Bharat Coking Coal Ltd,1.0,0.0,1.0
...,...,...,...,...,...
905,T1160,Bothra Group,0.0,0.0,0.0
906,T1167,JSW Infrastructure Ltd.,0.0,0.0,0.0
907,T1432,Maharashtra Maritime Board,0.0,0.0,0.0
908,T1382,Maharashtra Maritime Board,0.0,0.0,0.0


In [40]:
# In the 'firms' sheet, we don't need the share data, so remove them here
# Remove percentage patterns inside square brackets or parentheses
ownership_flat['firm'] = ownership_flat['firm'].str.replace(r'\[.*?\]', '', regex=True)  # Remove square brackets content
ownership_flat['firm'] = ownership_flat['firm'].str.replace(r'\(\d+(\.\d+)?%\)', '', regex=True)  # Remove percentage in parentheses
# Remove a dot at the end of the name (if it exists)
ownership_flat['firm'] = ownership_flat['firm'].str.replace(r'\.$', '', regex=True)
ownership_flat['firm'] = ownership_flat['firm'].str.strip()
ownership_flat

,asset_id,firm,share1,share2,share
0,M0464,Western Coalfields Ltd,1.0,0.0,1.0
1,M0465,The Singareni Collieries Co Ltd,1.0,0.0,1.0
2,M0466,Central Coalfields Ltd,1.0,0.0,1.0
3,M0467,South Eastern Coalfields Ltd,1.0,0.0,1.0
4,M0468,Bharat Coking Coal Ltd,1.0,0.0,1.0
...,...,...,...,...,...
905,T1160,Bothra Group,0.0,0.0,0.0
906,T1167,JSW Infrastructure Ltd,0.0,0.0,0.0
907,T1432,Maharashtra Maritime Board,0.0,0.0,0.0
908,T1382,Maharashtra Maritime Board,0.0,0.0,0.0


In [41]:
# Standardize firm name
# Function to standardize company names
def standardize_company_name(name):
    # Remove "Ltd" and "Limited"
    name = re.sub(r'\bLtd\b', '', name)
    name = re.sub(r'\bLimited\b', '', name)
    
    # Replace "Co" with "Company"
    name = re.sub(r'\bCo\b', 'Company', name)
    
    # Replace "Corp" with "Corporation"
    name = re.sub(r'\bCorp\b', 'Corporation', name)
    
    # Remove duplicate "Company Company"
    name = re.sub(r'\bCompany\b\s*\bCompany\b', 'Company', name)
    
    # Remove any extra spaces
    name = name.strip()
    
    return name

# Apply the function to the 'Company Name' column
ownership_flat['firm'] = ownership_flat['firm'].apply(standardize_company_name)

In [42]:
# Manually replace company names
#ownership_flat['firm'] = ownership_flat['firm'].replace('Gujarat Mineral Development Company', 'Gujarat Mineral Development Corporation')
ownership_flat['firm'] = ownership_flat['firm'].replace('Kakinada Seaport', 'Kakinada Seaports')

In [43]:
ownership_flat

,asset_id,firm,share1,share2,share
0,M0464,Western Coalfields,1.0,0.0,1.0
1,M0465,The Singareni Collieries Company,1.0,0.0,1.0
2,M0466,Central Coalfields,1.0,0.0,1.0
3,M0467,South Eastern Coalfields,1.0,0.0,1.0
4,M0468,Bharat Coking Coal,1.0,0.0,1.0
...,...,...,...,...,...
905,T1160,Bothra Group,0.0,0.0,0.0
906,T1167,JSW Infrastructure,0.0,0.0,0.0
907,T1432,Maharashtra Maritime Board,0.0,0.0,0.0
908,T1382,Maharashtra Maritime Board,0.0,0.0,0.0


In [44]:
merged = pd.merge(ownership_flat, unique_firms, how='left', left_on='firm', right_on='name')

# Drop unnecessary columns (optional) and select the desired ones
final_ownership = merged[['asset_id', 'firm_id', 'share']]
final_ownership # in the final sheet, 'share' == 0 means there's no share data

,asset_id,firm_id,share
0,M0464,F278,1.0
1,M0465,F262,1.0
2,M0466,F056,1.0
3,M0467,F238,1.0
4,M0468,F041,1.0
...,...,...,...
905,T1160,F050,0.0
906,T1167,F119,0.0
907,T1432,F156,0.0
908,T1382,F156,0.0


There are some conflicts that the same asset and firm have different share data, we leave it like this for now and will disuss it later

In [45]:
# Check for duplicates based on 'asset_id' and 'firm'
duplicates = ownership_flat.duplicated(subset=['asset_id', 'firm'], keep=False)

# Filter and display duplicate rows
df_duplicates = ownership_flat[duplicates]

# Show results
df_duplicates


,asset_id,firm,share1,share2,share
705,L100000102353,Madhya Pradesh Power Generating Company,0.0,1.00,1.00
706,L100000102353,Madhya Pradesh Power Generating Company,0.0,0.51,0.51
762,L100000102603,Telangana State Power Generation Corporation,0.0,1.00,1.00
763,L100000102603,Telangana State Power Generation Corporation,0.0,0.50,0.50


In [46]:
with pd.ExcelWriter('./Data/coal_supply_chain_india.xlsx') as writer:
    total_assets.to_excel(writer, sheet_name='assets', index=False)
    unique_firms.to_excel(writer, sheet_name='firms', index=False)
    final_ownership.to_excel(writer, sheet_name='ownership', index=False)